In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: alexisjihyeross. Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd 
import wandb
import numpy as np
import json

api = wandb.Api(timeout=50)
entity, project = "lm-informants", "0804_english"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [4]:
from tqdm import tqdm

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run_idx, run in tqdm(enumerate(runs), total=len(runs)):
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        'strategy_used_is_train',
        ]
                    
    if 'Model Eval Logs' in run.summary:
        model_eval_logs_exists = True
    else:
        model_eval_logs_exists = False
    
    # download Model Eval Logs table
    if model_eval_logs_exists:
        
        path = run.summary['Model Eval Logs']['path']
        run.file(path).download(replace=True)

        table = json.load(open(path))
        
        model_eval_logs = pd.DataFrame(data=table['data'], columns=table['columns'])
#         display(df)
#         table = run.use_artifact(f"run-{run.id}-Model Eval Logs").get("Model Eval Logs")

#         model_eval_logs = run.summary['Model Eval Logs']
#         model_eval_logs = (run.history(keys=['Model Eval Logs'])['Model Eval Logs'][0])
#         print(model_eval_logs)
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run (not finished): ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # tells us whether strategy_used_is_train_exists was logged as a metric
    strategy_used_is_train_exists = ('strategy_used_is_train' in history_df.columns)
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})

    num_steps = len(exp[keys[0]])
    for k in keys:
        if isinstance(exp[k], float) and np.isnan(exp[k]):
            print(f'Warning: key {k} is nan for run: ', run.path)
            # set to list of nans
            exp[k] = [np.nan] * num_steps
            continue
        # only make sure that the non-nan keys have num_steps many entries
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        r.update({'tags': run.tags})
        
        if model_eval_logs_exists:
            table_row = model_eval_logs.iloc[int(r['step'])]
            strategy_used = table_row['strategy_for_this_candidate']
            r.update({'strategy_used': strategy_used})
            
            if strategy_used_is_train_exists:
                assert r['strategy_used_is_train'] == (strategy_used == 'train'), (f"logged strategy_used_is_train is {r['strategy_used_is_train']}"
                                                                              f" but strategy_used from table is {strategy_used}"
                                                                              f" for step {r['step']}")
            
            # if strategy_used_is_train wasn't logged, set it with the table logged at the end
            else:
                r['strategy_used_is_train'] = (strategy_used == 'train')
        
        # strategy_used_is_train will be nan if BOTH strategy_used_is_train wasn't directly logged AND Model Eval Logs wasn't logged (it defaults to nan if not in the columns when looping through keys)
        # strategy_used will be nan if model_eval_logs_exists = False (i.e. the Model Eval Logs table wasn't logged)
        else:
            r['strategy_used'] = np.nan
        
        
            
                
    all_data.extend(results)
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


  1%|▊                                                                              | 1/92 [00:00<01:25,  1.06it/s]

filtering run (not finished):  ['lm-informants', '0804_english', 'zlvm6tp3']


  2%|█▋                                                                             | 2/92 [00:02<01:33,  1.04s/it]

filtering run (not finished):  ['lm-informants', '0804_english', 'pqzekt0a']


  7%|█████▏                                                                         | 6/92 [00:09<02:11,  1.53s/it]

filtering run (not finished):  ['lm-informants', '0804_english', 'jldl2t6n']


 50%|███████████████████████████████████████                                       | 46/92 [01:43<01:22,  1.80s/it]

filtering run (not finished):  ['lm-informants', '0804_english', 'ltr6vi0w']


100%|██████████████████████████████████████████████████████████████████████████████| 92/92 [03:34<00:00,  2.33s/it]


In [6]:
out_path = './0804_english.csv'
mega_df.to_csv(out_path)

In [5]:
display(mega_df)

,step,auc,strategy_used_is_train,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/num_candidates,config/log_log_alpha_ratio,config/max_updates_observe,config/max_updates_propose,strategy,wandb_id,tags,strategy_used
0,0.0,0.502481,1.0,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.995411,NaN,NaN,train,"[lm-informants, 0804_english, 9uy0a9k4]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
1,1.0,0.532777,1.0,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.995411,NaN,NaN,train,"[lm-informants, 0804_english, 9uy0a9k4]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
2,2.0,0.560588,1.0,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.995411,NaN,NaN,train,"[lm-informants, 0804_english, 9uy0a9k4]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
3,3.0,0.572590,1.0,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.995411,NaN,NaN,train,"[lm-informants, 0804_english, 9uy0a9k4]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
4,4.0,0.610302,1.0,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.995411,NaN,NaN,train,"[lm-informants, 0804_english, 9uy0a9k4]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21195,195.0,0.772892,1.0,0,0,train,0.000002,0.001385,False,english,None,symmetric,50,3.373793,1.0,NaN,train,"[lm-informants, 0804_english, lnrxr82p]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
21196,196.0,0.771735,1.0,0,0,train,0.000002,0.001385,False,english,None,symmetric,50,3.373793,1.0,NaN,train,"[lm-informants, 0804_english, lnrxr82p]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
21197,197.0,0.771921,1.0,0,0,train,0.000002,0.001385,False,english,None,symmetric,50,3.373793,1.0,NaN,train,"[lm-informants, 0804_english, lnrxr82p]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
21198,198.0,0.772132,1.0,0,0,train,0.000002,0.001385,False,english,None,symmetric,50,3.373793,1.0,NaN,train,"[lm-informants, 0804_english, lnrxr82p]","[0804_hyperparam_sweep, return_true_if_lexicon]",train
